# Excel Row Count Comparison Analysis

This notebook analyzes the Excel file to compare SLTLoadedRowCount and NewLakeLoadedRowCount values across all sheets.


In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import multiprocessing as mp
from functools import partial
import time

warnings.filterwarnings("ignore")

# Set up file path
excel_file = "./output/output_csvs.xlsx"
print(f"Analyzing file: {excel_file}")

# Set up multiprocessing
num_processes = min(mp.cpu_count(), 8)  # Use up to 8 cores, but not more than available
print(f"Using {num_processes} processes for parallel analysis")

Analyzing file: ./output/output_csvs.xlsx
Using 8 processes for parallel analysis


In [14]:
# Read the Excel file and get all sheet names
try:
    excel_file_obj = pd.ExcelFile(excel_file)
    sheet_names = excel_file_obj.sheet_names
    print(f"Found {len(sheet_names)} sheets: {sheet_names}")
except Exception as e:
    print(f"Error reading Excel file: {e}")
    sheet_names = []

Found 969 sheets: ['a016', 'a068', 'a551', 'a594', 'a651', 'a652', 'a653', 'a654', 'a655', 'a656', 'a717', 'a733', 'a734', 'a735', 'a736', 'a737', 'a738', 'a748', 'a749', 'a753', 'a757', 'a762', 'acdoca', 'acdoca_m_extract', 'acdocp', 'adr12', 'adr2', 'adr3', 'adr6', 'adrc', 'adrct', 'adrp', 'adrt', 'afih', 'afko', 'afpo', 'afru', 'afvc', 'afvu', 'afvv', 'agr_1251', 'agr_tcodes', 'agr_users', 'anka', 'ankb', 'ankt', 'anla', 'anlb', 'anlc', 'anlh', 'anli', 'anlz', 'auak', 'auas', 'aufk', 'aufm', 'ausp', 'bic_azco_d0021', 'bic_azco_d0022', 'bic_azpf_d0011', 'bic_azpf_d0012', 'bic_azpf_d0052', 'bic_hzglaccnt', 'bic_pzconsview', 'bic_pzdatasrc', 'bic_pzversion', 'bic_szflow', 'bic_tzconsview', 'bic_tzdatasrc', 'bic_tzpyear', 'bic_tzversion', 'bkpf', 'bp1010', 'bseg', 'bset', 'but000', 'but050', 'c001', 'c002', 'c003', 'c004', 'c005', 'c501', 'cabn', 'cabnt', 'cawn', 'cawnt', 'cdhdr', 'cdpos', 'ce9tysn_h', 'ce9tysn_ic', 'ce9tysn_iq', 'ce9tysn_iv', 'cepc', 'cepct', 'ckis', 'ckmlcr', 'ckmlhd'

In [15]:
# Initialize variables to store results
all_discrepancies = []
sheets_with_discrepancies = 0
total_sheets_analyzed = 0
total_rows_analyzed = 0
total_discrepancies = 0

print("Starting analysis of all sheets...")
print("=" * 50)

Starting analysis of all sheets...


In [16]:
# Analyze each sheet
for sheet_name in sheet_names:
    print(f"\nAnalyzing sheet: {sheet_name}")
    print("-" * 30)

    try:
        # Read the sheet
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        total_sheets_analyzed += 1

        print(f"Columns found: {list(df.columns)}")
        print(f"Total rows in sheet: {len(df)}")

        # Skip if the sheet is empty
        if df.empty:
            print(f"⚠️  Skipping empty sheet: {sheet_name}")
            continue

        # Check if required columns exist
        slt_col = None
        newlake_col = None

        # Look for columns that might contain the row counts (case-insensitive)
        for col in df.columns:
            col_lower = str(col).lower()
            if "sltloadedrowcount" in col_lower:
                slt_col = col
            elif "newlakeloadedrowcount" in col_lower:
                newlake_col = col

        if slt_col is None or newlake_col is None:
            print(f"⚠️  Missing required columns in {sheet_name}")
            print(f"   SLTLoadedRowCount column: {slt_col}")
            print(f"   NewLakeLoadedRowCount column: {newlake_col}")
            continue

        print(f"   Using columns: '{slt_col}' and '{newlake_col}'")

        # Convert to numeric, handling any non-numeric values
        df[slt_col] = pd.to_numeric(df[slt_col], errors="coerce")
        df[newlake_col] = pd.to_numeric(df[newlake_col], errors="coerce")

        # Find rows where counts don't match
        mask = (
            (df[slt_col] != df[newlake_col])
            & (~df[slt_col].isna())
            & (~df[newlake_col].isna())
        )
        discrepancies = df[mask].copy()

        if len(discrepancies) > 0:
            sheets_with_discrepancies += 1
            total_discrepancies += len(discrepancies)

            print(f"❌ Found {len(discrepancies)} discrepancies in {sheet_name}")

            # Add sheet name to each discrepancy row
            discrepancies["SheetName"] = sheet_name
            discrepancies["SLTCount"] = discrepancies[slt_col]
            discrepancies["NewLakeCount"] = discrepancies[newlake_col]
            discrepancies["Difference"] = (
                discrepancies[newlake_col] - discrepancies[slt_col]
            )

            all_discrepancies.append(discrepancies)

            # Show first few discrepancies
            print("   First few discrepancies:")
            display_cols = ["SheetName", "SLTCount", "NewLakeCount", "Difference"] + [
                col for col in df.columns if col not in [slt_col, newlake_col]
            ]
            print(discrepancies[display_cols].head().to_string(index=False))
        else:
            print(f"✅ No discrepancies found in {sheet_name}")

        total_rows_analyzed += len(df)

    except Exception as e:
        print(f"❌ Error processing sheet {sheet_name}: {e}")

print(f"\n{'='*50}")
print("Analysis Complete!")


Analyzing sheet: a016
------------------------------
Columns found: ['FromEpochMicro', 'ToEpochMicro', 'SLTLoadedRowCount', 'NewLakeLoadedRowCount', 'JobID', 'JobStartEpochMicro', 'JobEndEpochMicro']
Total rows in sheet: 70
   Using columns: 'SLTLoadedRowCount' and 'NewLakeLoadedRowCount'
✅ No discrepancies found in a016

Analyzing sheet: a068
------------------------------
Columns found: ['FromEpochMicro', 'ToEpochMicro', 'SLTLoadedRowCount', 'NewLakeLoadedRowCount', 'JobID', 'JobStartEpochMicro', 'JobEndEpochMicro']
Total rows in sheet: 6
   Using columns: 'SLTLoadedRowCount' and 'NewLakeLoadedRowCount'
✅ No discrepancies found in a068

Analyzing sheet: a551
------------------------------
Columns found: ['FromEpochMicro', 'ToEpochMicro', 'SLTLoadedRowCount', 'NewLakeLoadedRowCount', 'JobID', 'JobStartEpochMicro', 'JobEndEpochMicro']
Total rows in sheet: 63
   Using columns: 'SLTLoadedRowCount' and 'NewLakeLoadedRowCount'
✅ No discrepancies found in a551

Analyzing sheet: a594
------

In [17]:
# Summary statistics # type: ignore
print("\nSUMMARY STATISTICS")
print("=" * 50)
print(f"Total sheets analyzed: {total_sheets_analyzed}")
print(f"Total rows analyzed: {total_rows_analyzed}")
print(f"Sheets with discrepancies: {sheets_with_discrepancies}")
print(f"Total discrepancies found: {total_discrepancies}")
# print(f"Processing time: {processing_time:.2f} seconds")

if total_rows_analyzed > 0:
    # print(f"Processing speed: {total_rows_analyzed/processing_time:.0f} rows/second")
    if total_discrepancies > 0:
        print(f"Discrepancy rate: {(total_discrepancies/total_rows_analyzed)*100:.2f}%")
    else:
        print("\n🎉 All row counts match perfectly!")
else:
    print("\n⚠️ No data was processed!")


SUMMARY STATISTICS
Total sheets analyzed: 969
Total rows analyzed: 152799
Sheets with discrepancies: 1
Total discrepancies found: 101
Discrepancy rate: 0.07%


In [18]:
# Display all discrepancies if any found
if all_discrepancies:
    print("\nALL DISCREPANCIES DETAILS")
    print("=" * 50)

    # Combine all discrepancies
    combined_discrepancies = pd.concat(all_discrepancies, ignore_index=True)

    # Display all discrepancies
    display_cols = ["SheetName", "SLTCount", "NewLakeCount", "Difference"] + [
        col
        for col in combined_discrepancies.columns
        if col not in ["SheetName", "SLTCount", "NewLakeCount", "Difference"]
    ]

    print(f"\nShowing all {len(combined_discrepancies)} discrepancies:")
    print(combined_discrepancies[display_cols].to_string(index=False))

    # Save detailed results to CSV for further analysis
    output_file = "discrepancy_details.csv"
    combined_discrepancies.to_csv(output_file, index=False)
    print(f"\nDetailed results saved to: {output_file}")
else:
    print("\nNo discrepancies found - all row counts match!")
    combined_discrepancies = pd.DataFrame()


ALL DISCREPANCIES DETAILS

Showing all 101 discrepancies:
SheetName  SLTCount  NewLakeCount  Difference      FromEpochMicro        ToEpochMicro  SLTLoadedRowCount  NewLakeLoadedRowCount                                JobID  JobStartEpochMicro    JobEndEpochMicro
     kna1         0             1           1 2025-10-02 21:19:44 2025-10-02 21:24:58                  0                      1 e5bc45d7-0934-4687-9b63-1c7f19b10b08 2025-10-02 21:30:35 2025-10-02 21:30:39
     kna1         0             1           1 2025-10-02 21:14:00 2025-10-02 21:19:44                  0                      1 54a3cfb4-7dce-4c14-81c4-34a934f83814 2025-10-02 21:25:35 2025-10-02 21:25:38
     kna1         0             1           1 2025-10-02 21:13:15 2025-10-02 21:14:00                  0                      1 1a24fb3d-80d5-4b1b-94d0-c3cf9b7698c9 2025-10-02 21:20:34 2025-10-02 21:20:37
     kna1         0             2           2 2025-10-02 21:07:32 2025-10-02 21:13:15                  0                 

In [22]:
# Generate summary report # type: ignore
report_content = f"""# Excel Row Count Comparison Report

## Analysis Summary

**File Analyzed:** {excel_file}  
**Analysis Date:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}  
**Processing Method:** Parallel analysis using {num_processes} processes

### Statistics
- Total sheets analyzed: {total_sheets_analyzed}
- Total rows analyzed: {total_rows_analyzed}
- Sheets with discrepancies: {sheets_with_discrepancies}
- Total discrepancies found: {total_discrepancies}

### Discrepancy Rate
{(total_discrepancies/total_rows_analyzed)*100:.2f}% of rows have mismatched counts

## Detailed Results

"""

if total_discrepancies > 0:  # type: ignore
    report_content += f"""
### All Discrepancies Found

The following {total_discrepancies} rows have mismatched SLTLoadedRowCount and NewLakeLoadedRowCount values:

"""

    # Add each discrepancy as a table
    for i, discrepancy in enumerate(all_discrepancies):
        sheet_name = discrepancy["SheetName"].iloc[0]
        report_content += (
            f"\n#### Sheet: {sheet_name} ({len(discrepancy)} discrepancies)\n\n"
        )

        # Create a markdown table for this sheet's discrepancies
        display_cols = ["SLTCount", "NewLakeCount", "Difference"] + [
            col
            for col in discrepancy.columns
            if col not in ["SheetName", "SLTCount", "NewLakeCount", "Difference"]
        ]

        # Add table header
        report_content += "| " + " | ".join(display_cols) + " |\n"
        report_content += "| " + " | ".join(["---"] * len(display_cols)) + " |\n"

        # Add each row
        for _, row in discrepancy.iterrows():
            row_data = []
            for col in display_cols:
                value = row[col]
                if pd.isna(value):
                    row_data.append("N/A")
                else:
                    row_data.append(str(value))
            report_content += "| " + " | ".join(row_data) + " |\n"

        report_content += "\n"
else:
    report_content += (
        "\n**✅ No discrepancies found - all row counts match perfectly!**\n"
    )

report_content += (
    f"\n---\n\n*Report generated on {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}*"
)

# Save the report
with open("discrepancy_report.md", "w") as f:
    f.write(report_content)

print(f"\nSummary report saved to: discrepancy_report.md")
print("\nReport preview:")
print("=" * 50)
print(report_content[:1000] + "..." if len(report_content) > 1000 else report_content)


Summary report saved to: discrepancy_report.md

Report preview:
# Excel Row Count Comparison Report

## Analysis Summary

**File Analyzed:** ./output/output_csvs.xlsx  
**Analysis Date:** 2025-10-15 14:15:27  
**Processing Method:** Parallel analysis using 8 processes

### Statistics
- Total sheets analyzed: 969
- Total rows analyzed: 152799
- Sheets with discrepancies: 1
- Total discrepancies found: 101

### Discrepancy Rate
0.07% of rows have mismatched counts

## Detailed Results


### All Discrepancies Found

The following 101 rows have mismatched SLTLoadedRowCount and NewLakeLoadedRowCount values:


#### Sheet: kna1 (101 discrepancies)

| SLTCount | NewLakeCount | Difference | FromEpochMicro | ToEpochMicro | SLTLoadedRowCount | NewLakeLoadedRowCount | JobID | JobStartEpochMicro | JobEndEpochMicro |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 0 | 1 | 1 | 2025-10-02 21:19:44 | 2025-10-02 21:24:58 | 0 | 1 | e5bc45d7-0934-4687-9b63-1c7f19b10b08 | 2025-10-02 21:30:

In [20]:
# Performance analysis and optimization notes # type: ignore
print("\nPERFORMANCE ANALYSIS")
print("=" * 50)
print(f"CPU cores available: {mp.cpu_count()}")
print(f"Processes used: {num_processes}")
print(f"Total processing time: {processing_time:.2f} seconds")

if total_sheets_analyzed > 0:
    avg_time_per_sheet = processing_time / total_sheets_analyzed
    print(f"Average time per sheet: {avg_time_per_sheet:.2f} seconds")

    # Estimate sequential processing time
    estimated_sequential_time = avg_time_per_sheet * total_sheets_analyzed
    speedup = estimated_sequential_time / processing_time if processing_time > 0 else 1
    print(f"Estimated sequential time: {estimated_sequential_time:.2f} seconds")
    print(f"Speedup factor: {speedup:.1f}x faster with parallel processing")

print(f"\nMemory usage optimization:")
print(f"- Each process handles one sheet at a time")
print(f"- Results are collected and processed after parallel execution")
print(f"- Large DataFrames are processed in separate processes to avoid memory issues")


PERFORMANCE ANALYSIS
CPU cores available: 10
Processes used: 8


NameError: name 'processing_time' is not defined

In [ ]:
# Alternative: Simple parallel processing using concurrent.futures (more reliable in Jupyter) # type: ignore
from concurrent.futures import ProcessPoolExecutor, as_completed
import sys


def run_parallel_analysis_alternative():
    """Alternative parallel processing using concurrent.futures"""
    print(f"\nTrying alternative parallel processing method...")
    print(f"Processing {len(sheet_names)} sheets using {num_processes} processes...")

    results = []

    try:
        with ProcessPoolExecutor(max_workers=num_processes) as executor:
            # Submit all tasks
            future_to_sheet = {
                executor.submit(analyze_sheet, sheet_name, excel_file): sheet_name
                for sheet_name in sheet_names
            }

            # Collect results as they complete
            for i, future in enumerate(as_completed(future_to_sheet)):
                sheet_name = future_to_sheet[future]
                try:
                    result = future.result()
                    results.append(result)
                    if i % 50 == 0:  # Progress indicator
                        print(f"Completed {i+1}/{len(sheet_names)} sheets")
                except Exception as e:
                    print(f"Error processing {sheet_name}: {e}")
                    # Add error result
                    results.append(
                        {
                            "sheet_name": sheet_name,
                            "success": False,
                            "error": str(e),
                            "columns": [],
                            "total_rows": 0,
                            "discrepancies": pd.DataFrame(),
                            "slt_col": None,
                            "newlake_col": None,
                        }
                    )

        print("✅ Alternative parallel processing completed successfully")
        return results

    except Exception as e:
        print(f"⚠️ Alternative parallel processing also failed: {e}")
        return None


# Uncomment the line below to try the alternative method if the main one fails
# results = run_parallel_analysis_alternative()

In [ ]:
# Test multiprocessing setup with a small sample # type: ignore
def test_multiprocessing():
    """Test if multiprocessing works with a small sample"""
    print("Testing multiprocessing setup...")

    # Test with just the first 3 sheets
    test_sheets = sheet_names[:3] if len(sheet_names) >= 3 else sheet_names

    try:
        analyze_func = partial(analyze_sheet, excel_file=excel_file)
        with mp.Pool(processes=min(2, num_processes)) as pool:
            test_results = pool.map(analyze_func, test_sheets)

        print(
            f"✅ Multiprocessing test successful with {len(test_results)} test sheets"
        )
        return True

    except Exception as e:
        print(f"❌ Multiprocessing test failed: {e}")
        return False


# Run the test
multiprocessing_works = test_multiprocessing()

if not multiprocessing_works:
    print("\n⚠️ Multiprocessing is not working properly.")
    print("The notebook will use sequential processing instead.")
    print("This may be slower but will still complete the analysis.")